In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Load and transform the data
data = pd.read_csv('./data-01234/vns_dataset_threshold_sim.csv')

# Transform the data for AB fibers
data_ab = data[['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude', 'AB_activation_level']].copy()
data_ab.columns = ['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude', 'activation_level']
data_ab['fibre_type'] = 'AB'

# Transform the data for C fibers
data_c = data[['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude', 'C_activation_level']].copy()
data_c.columns = ['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude', 'activation_level']
data_c['fibre_type'] = 'C'

# Combine the transformed data
df_transformed = pd.concat([data_ab, data_c], ignore_index=True)

# Encode the categorical feature
label_encoder = LabelEncoder()
df_transformed['fibre_type'] = label_encoder.fit_transform(df_transformed['fibre_type'])

# Split the data
X = df_transformed[['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude', 'fibre_type']]
y = df_transformed[['activation_level']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the continuous features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude']])
X_test_scaled = scaler.transform(X_test[['nerve_a', 'nerve_b', 'pulse_width', 'frequency', 'amplitude']])

# Re-attach the categorical feature (fibre_type)
X_train_scaled = np.hstack((X_train_scaled, X_train[['fibre_type']].values))
X_test_scaled = np.hstack((X_test_scaled, X_test[['fibre_type']].values))

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\btari\AppData\Local\Temp\ipykernel_20756\724535008.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['nerve_a', 'nerve_b']] = scaler.fit_transform(X[['nerve_a', 'nerve_b']])


In [8]:
from keras import layers, losses, Model, Input
from keras import backend as K

# Hyperparameters
latent_dim = 2  # z_stimulation + z_other
hidden_dim = 128
epochs = 50
batch_size = 32

# Encoder
encoder_inputs = Input(shape=(X_train.shape[1],))  # Number of input features
x = layers.Dense(hidden_dim, activation="relu")(encoder_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

# Sampling Function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon
z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
latent_inputs = Input(shape=(latent_dim,))
decoder_inputs = layers.concatenate([latent_inputs, encoder_inputs])  # Include original inputs
x = layers.Dense(hidden_dim, activation="relu")(decoder_inputs)
decoder_outputs = layers.Dense(1, activation="linear")(x)  # Output activation level

# Models
encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
decoder = Model(latent_inputs, decoder_outputs, name="decoder")
vae = Model(encoder_inputs, decoder(encoder(encoder_inputs)[2]), name="vae")

# VAE Loss (Custom Function)
def vae_loss(x, x_decoded_mean):
    reconstruction_loss = losses.mean_squared_error(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(reconstruction_loss + kl_loss)

# Compile and Train
vae.compile(optimizer="adam", loss=vae_loss)
vae.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, X_test))


ValueError: setting an array element with a sequence.

In [ ]:

# Sampling function
def sample_for_activation_level(activation_level, nerve_a, nerve_b, fibre_type, latent_dim):
    z_sample = np.random.normal(size=(1, latent_dim))
    activation_level_tensor = np.array(activation_level).reshape(1, -1)
    nerve_a_tensor = np.array(nerve_a).reshape(1, -1)
    nerve_b_tensor = np.array(nerve_b).reshape(1, -1)
    fibre_type_tensor = np.array(fibre_type).reshape(1, -1)
    return decoder.predict(np.concatenate([z_sample, activation_level_tensor, nerve_a_tensor, nerve_b_tensor, fibre_type_tensor], axis=1))

# Example: Predict inputs for a given activation level A or B
activation_level_example = [0.5]  # For instance, specifying activation level A
nerve_a_example = [0.1]
nerve_b_example = [0.2]
fibre_type_example = [1]  # Assume 1 corresponds to a certain fibre type in the encoded label

predicted_inputs = sample_for_activation_level(activation_level_example, nerve_a_example, nerve_b_example, fibre_type_example, latent_dim)
print(predicted_inputs)
